In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone

Cloning into 'yolov5'...
remote: Enumerating objects: 16575, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 16575 (delta 28), reused 37 (delta 18), pack-reused 16522
Receiving objects: 100% (16575/16575), 15.03 MiB | 31.87 MiB/s, done.
Resolving deltas: 100% (11387/11387), done.


In [3]:
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 25.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 34.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 26.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 

In [4]:
import os
os.listdir('/kaggle/working/yolov5')

['.gitattributes',
 'data',
 'LICENSE',
 'detect.py',
 'README.zh-CN.md',
 'benchmarks.py',
 'train.py',
 'models',
 '.gitignore',
 'utils',
 'pyproject.toml',
 'segment',
 '.git',
 '.github',
 'export.py',
 'CITATION.cff',
 'tutorial.ipynb',
 'hubconf.py',
 'CONTRIBUTING.md',
 'val.py',
 'README.md',
 'classify',
 'requirements.txt',
 '.dockerignore']

In [5]:
from yolov5 import YOLOv5

In [6]:
#tiny-yolov5s

import numpy as np
from PIL import Image
from yolov5 import YOLOv5
import cv2

model = YOLOv5('yolov5s.pt')  # Make sure the model file path is correct.

def detect_and_crop(image_path):
    image = Image.open(image_path).convert("RGB")
    results = model.predict(np.array(image))
    detections = results.pandas().xyxy[0]  # Extract bounding boxes as DataFrame

    crops = []
    details = []
    image_np = np.array(image)  # Convert PIL image to numpy array

    for _, row in detections.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        conf = row['confidence']  # Extract confidence score
        class_id = row['class']  # Extract class ID
        class_name = row['name']  # Extract class name

        crop = image_np[y1:y2, x1:x2]
        crops.append(crop)
        details.append((class_id, class_name, conf, (x1, y1, x2, y2)))  # Append all details as a tuple

    return crops, details, image_np

image_path = '/kaggle/input/idai710/foggy_images/test/berlin_000000_000019_leftImg8bit.png'  # Ensure the image path is correct.


crops, details, preprocessed_image = detect_and_crop(image_path)

for idx, (crop, detail) in enumerate(zip(crops, details)):
    class_id, class_name, conf, bbox = detail
    print(f"Crop {idx+1}: Class ID={class_id}, Class Name={class_name}, Confidence={conf:.2f}, BBox={bbox}")
    save_path = f'crop_{idx + 1}_{class_name}_{int(conf * 100)}.png'  # Corrected file path format.
    cv2.imwrite(save_path, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))  # Ensure conversion from RGB to BGR for OpenCV.


Crop 1: Class ID=2, Class Name=car, Confidence=0.83, BBox=(1093, 422, 1189, 503)
Crop 2: Class ID=2, Class Name=car, Confidence=0.81, BBox=(2, 524, 215, 852)
Crop 3: Class ID=2, Class Name=car, Confidence=0.79, BBox=(833, 422, 964, 529)
Crop 4: Class ID=2, Class Name=car, Confidence=0.78, BBox=(723, 437, 819, 508)
Crop 5: Class ID=2, Class Name=car, Confidence=0.75, BBox=(948, 425, 1023, 490)
Crop 6: Class ID=5, Class Name=bus, Confidence=0.73, BBox=(0, 316, 240, 529)
Crop 7: Class ID=2, Class Name=car, Confidence=0.64, BBox=(295, 438, 448, 501)
Crop 8: Class ID=2, Class Name=car, Confidence=0.57, BBox=(1949, 344, 2047, 501)
Crop 9: Class ID=2, Class Name=car, Confidence=0.42, BBox=(0, 462, 119, 561)
Crop 10: Class ID=9, Class Name=traffic light, Confidence=0.40, BBox=(1348, 343, 1374, 385)
Crop 11: Class ID=2, Class Name=car, Confidence=0.39, BBox=(798, 405, 868, 468)
Crop 12: Class ID=2, Class Name=car, Confidence=0.37, BBox=(1763, 422, 1845, 472)
Crop 13: Class ID=56, Class Name=cha

In [ ]:
#DCDPN:
#new_version: 

import os
from PIL import Image
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from tqdm import tqdm  
import os




class DehazingDataset(Dataset):
    def __init__(self, root_dir, subset='train', transform=None):
        self.root_dir = root_dir
        self.subset = subset
        self.transform = transform
        self.foggy_path = os.path.join(root_dir, 'foggy_images', subset)
        self.original_path = os.path.join(root_dir, 'original_images', subset)
        
        # Updated to filter for .png files
        self.image_files = [f for f in os.listdir(self.foggy_path) if f.endswith('.png') and os.path.isfile(os.path.join(self.foggy_path, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        file_name = self.image_files[idx]
        foggy_image_path = os.path.join(self.foggy_path, file_name)
        original_image_path = os.path.join(self.original_path, file_name)

        foggy_image = Image.open(foggy_image_path).convert('RGB')
        original_image = Image.open(original_image_path).convert('RGB')

        if self.transform:
            foggy_image = self.transform(foggy_image)
            original_image = self.transform(original_image)

        return {'foggy': foggy_image, 'original': original_image}


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = DoubleConv(64, 128)
        self.down2 = DoubleConv(128, 256)
        self.down3 = DoubleConv(256, 512)
        self.down4 = DoubleConv(512, 512)
        self.up1 = DoubleConv(1024, 256)
        self.up2 = DoubleConv(512, 128)
        self.up3 = DoubleConv(256, 64)
        self.up4 = DoubleConv(128, 64)
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

        self.pool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.pool(x1)
        x2 = self.down1(x2)
        x3 = self.pool(x2)
        x3 = self.down2(x3)
        x4 = self.pool(x3)
        x4 = self.down3(x4)
        x5 = self.pool(x4)
        x5 = self.down4(x5)

        x = self.upsample(x5)
        x = torch.cat([x, x4], dim=1)
        x = self.up1(x)
        x = self.upsample(x)
        x = torch.cat([x, x3], dim=1)
        x = self.up2(x)
        x = self.upsample(x)
        x = torch.cat([x, x2], dim=1)
        x = self.up3(x)
        x = self.upsample(x)
        x = torch.cat([x, x1], dim=1)
        x = self.up4(x)
        x = self.outc(x)
        return x


import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

# Assuming DehazingDataset and UNet are defined/imported as above or from another module

root_dir = '/kaggle/input/idai710'
checkpoint_path = 'checkpoint.pth'

transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = DehazingDataset(root_dir=root_dir, subset='train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

val_dataset = DehazingDataset(root_dir=root_dir, subset='val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(n_channels=3, n_classes=3).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to load the checkpoint
def load_checkpoint(filepath):
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath)
        model.load_state_dict(checkpoint['model_state'])
        optimizer.load_state_dict(checkpoint['optimizer_state'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Loaded checkpoint from epoch {checkpoint['epoch']}")
        return start_epoch
    else:
        return 1

# Function to save the checkpoint
def save_checkpoint(state, filename='checkpoint.pth'):
    torch.save(state, filename)

num_epochs = 10
start_epoch = load_checkpoint(checkpoint_path)

for epoch in range(start_epoch, num_epochs + 1):
    model.train()
    running_loss = 0.0
    train_progress = tqdm(train_loader, desc=f'Training Epoch {epoch}/{num_epochs}', total=len(train_loader))
    for data in train_progress:
        inputs, labels = data['foggy'].to(device), data['original'].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        train_progress.set_postfix({'train_loss': loss.item()})

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch: {epoch}/{num_epochs}, Train Loss: {epoch_loss:.4f}')

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_progress = tqdm(val_loader, desc=f'Validation Epoch {epoch}/{num_epochs}', total=len(val_loader))
    with torch.no_grad():
        for data in val_progress:
            inputs, labels = data['foggy'].to(device), data['original'].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * inputs.size(0)
            val_progress.set_postfix({'val_loss': loss.item()})

    val_loss = val_running_loss / len(val_loader.dataset)
    print(f'Epoch {epoch}/{num_epochs}, Val Loss: {val_loss:.4f}')
    
    torch.save(model.state_dict(), 'dcdpn.pth')


    # Save checkpoint after every epoch
    save_checkpoint({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict()
    }, checkpoint_path)








In [8]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2
import os

# Import the YOLOv5 model from its module
from yolov5 import YOLOv5

# Initialize the YOLOv5 model with the specified weights
model = YOLOv5('yolov5x.pt')

def load_image(image_path):
    """Load an image, resize it, and pad it to a square of 640x640, maintaining aspect ratio."""
    image = Image.open(image_path).convert("RGB")
    original_width, original_height = image.size
    max_size = 640
    scale = min(max_size / original_width, max_size / original_height)
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)
    image = image.resize((new_width, new_height), Image.ANTIALIAS)
    new_image = Image.new('RGB', (max_size, max_size))
    new_image.paste(image, ((max_size - new_width) // 2, (max_size - new_height) // 2))
    return np.array(new_image)

def detect_objects(image_path):
    """Use the YOLO model to detect objects in an image."""
    image = load_image(image_path)
    results = model.predict(image)
    return results

def draw_boxes(image_np, detections):
    """Draw bounding boxes and labels on the image."""
    image = Image.fromarray(image_np)
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    for index, row in detections.iterrows():
        draw.rectangle([(row['xmin'], row['ymin']), (row['xmax'], row['ymax'])], outline="red", width=2)
        label = f"{row['name']} {row['confidence']:.2f}"
        draw.text((row['xmin'], row['ymin']), label, fill="red", font=font)
    return np.array(image)

def save_detections(image_path, results, output_folder):
    """Save the detection results as images with bounding boxes and as text files."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    image_np = load_image(image_path)
    detections = results.pandas().xyxy[0]
    image_with_boxes = draw_boxes(image_np, detections)
    output_image_path = os.path.join(output_folder, os.path.basename(image_path))
    Image.fromarray(image_with_boxes).save(output_image_path)
    output_text_path = os.path.join(output_folder, os.path.splitext(os.path.basename(image_path))[0] + '_detections.txt')
    with open(output_text_path, 'w') as f:
        for index, row in detections.iterrows():
            f.write(f"Class ID: {row['class']}, Class Name: {row['name']}, Confidence: {row['confidence']:.2f}, BBox: [{row['xmin']}, {row['ymin']}, {row['xmax']}, {row['ymax']}]\n")
    print(f"Results saved: {output_image_path} and {output_text_path}")

# Usage example
image_path = '/kaggle/input/idai710/foggy_images/test/berlin_000000_000019_leftImg8bit.png'
output_folder = 'detected_results'
results = detect_objects(image_path)
save_detections(image_path, results, output_folder)


/tmp/ipykernel_34/3698935993.py:20: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize((new_width, new_height), Image.ANTIALIAS)
/tmp/ipykernel_34/3698935993.py:20: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image = image.resize((new_width, new_height), Image.ANTIALIAS)


Results saved: detected_results/berlin_000000_000019_leftImg8bit.png and detected_results/berlin_000000_000019_leftImg8bit_detections.txt


In [15]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
#from dataloader import DehazingDataset
#from aodnet import DehazeNetAttention
from utils import ssim
import numpy as np
from torchvision.utils import save_image

def collate_fn(batch):
    batch_mod = {key: [d[key] for d in batch] for key in batch[0]}
    batch_mod['original_image'] = torch.stack(batch_mod['original_image'])
    batch_mod['foggy_image'] = torch.stack(batch_mod['foggy_image'])
    return batch_mod

original_dir = '/kaggle/input/idai710/original_images'
foggy_dir = '/kaggle/input/idai710/foggy_images'
bb_dir = '/kaggle/input/ann-ds/ann_ds'

save_dir = '/home/stu12/s11/ak1825/idai710/Project/Dehazed_Images'
os.makedirs(save_dir, exist_ok=True) 

transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])

test_dataset = DehazingDataset(original_dir, foggy_dir, bb_dir, 'test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=collate_fn)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

try:
    model = torch.load('/kaggle/input/dcdpn-path/dcdpn_path.ptht').to(device)
    model.eval()
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

loss_fn = nn.MSELoss()
running_loss = 0.0
total_ssim = 0.0
num_samples = 0

with torch.no_grad():
    for data in test_loader:
        original_images = data['original_image'].to(device)
        foggy_images = data['foggy_image'].to(device)
        filenames = data['filename']  # Extract filenames from the dataset
        bounding_boxes = data['bboxes']

        outputs = model(foggy_images, bounding_boxes)  # Assuming the model does not need bounding boxes
        loss = loss_fn(outputs, original_images)

        outputs_np = outputs.permute(0, 2, 3, 1).cpu().numpy()
        original_images_np = original_images.permute(0, 2, 3, 1).cpu().numpy()

        batch_ssim = np.mean([ssim(out, orig) for out, orig in zip(outputs_np, original_images_np)])

        running_loss += loss.item() * original_images.size(0)
        total_ssim += batch_ssim * original_images.size(0)
        num_samples += original_images.size(0)

        # Save each output image using the original filename
        for output, filename in zip(outputs, filenames):
            save_path = os.path.join(save_dir, filename)  # Use the original filename
            save_image(output, save_path)

avg_loss = running_loss / num_samples
avg_ssim = total_ssim / num_samples
print(f"Average Loss: {avg_loss:.4f}")
print(f"Average SSIM: {avg_ssim:.4f}")

ImportError: cannot import name 'ssim' from 'utils' (/opt/conda/lib/python3.10/site-packages/yolov5/utils/__init__.py)